<a href="https://colab.research.google.com/github/githubramkiran/LanggraphAgent/blob/main/Copy_of_langgraph_RAG_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU "langchain-chroma>=0.1.2" langchain langchain-openai langgraph
!pip install -qU  langchain-google-genai langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 10.2 MB/s eta

In [ ]:
import os,getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your openai API key: ")
#os.environ["LANGSMITH_TRACING"] = "true"

Enter your openai API key: ··········


In [ ]:
from langchain_community.embeddings import FakeEmbeddings
embeddings = FakeEmbeddings(size=50)
vector = embeddings.embed_query("hello, world!")
vector[:5]

[np.float64(0.7614938460318729),
 np.float64(0.5901773488152734),
 np.float64(-0.5226592810013052),
 np.float64(-1.2221518111687573),
 np.float64(-0.5251848524518787)]

In [ ]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",
)

In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
    id=1,
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
    id=2,
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
    id=3,
)

documents = [
    document_1,
    document_2,
    document_3,
   ]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['22a260b2-ad2a-490d-b748-231bb527905e',
 'ac01bf12-511c-403d-aa53-d4b9378849bc',
 'ffd443ad-e617-46b3-9c62-b5dce0c63131']

In [ ]:
retriever = vector_store.as_retriever(
    search_type="mmr", search_kwargs={"k": 1, "fetch_k": 5}
)
retriever.invoke("langchain", filter={"source": "news"})

[Document(id='ac01bf12-511c-403d-aa53-d4b9378849bc', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.')]

In [ ]:

from langchain.messages import AnyMessage
from typing_extensions import TypedDict, Annotated
import operator


class MessagesState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]
    llm_calls: int
    query:str
    context:str

# Step 3: Define model node
from langchain.messages import SystemMessage

#from langgraph.nodes import LLMNode
from langgraph.prebuilt import ToolNode
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain.tools import tool
from langchain.chat_models import init_chat_model

#@tool
def retriever_node(state: dict) -> str:
 response=retriever.invoke(query)
 print('retriver node response:',response)
 context=response[0]
 return {"context":context}
# 2. Define nodes

#tools = [retriever_node]
#tool_node = ToolNode(tools)
from langchain.messages import HumanMessage


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
def llm_call(state: dict):
    """LLM decides whether to call a tool or not"""
    print('llm call with state',query)
    return {
        "messages": [
            llm.invoke(
                [HumanMessage(content=query),
                    SystemMessage(
                        content="You are a helpful assistant. Use the following context in your response:{context}"
                    )
                ]
                + state["messages"]
            )
        ],
        "llm_calls": state.get('llm_calls', 0) + 1
    }
# 3. Build graph
#graph = Graph()
agent_builder  = StateGraph(MessagesState)
agent_builder .add_node("llm_call", llm_call)
agent_builder .add_node("tool_node", retriever_node)

# Connect retriever → generator

# Add edges to connect nodes
agent_builder .add_edge(START, "tool_node")#retriver
agent_builder .add_edge("tool_node", "llm_call")#generator
agent_builder .add_edge("llm_call",END)
agent = agent_builder.compile()
# 4. Run workflow
query = "What is langchain?"
from langchain.messages import HumanMessage
messages = [HumanMessage(content=query)]
messages = agent.invoke({"messages": messages})
for m in messages["messages"]:
    m.pretty_print()


retriver node response: [Document(id='ac01bf12-511c-403d-aa53-d4b9378849bc', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.')]
llm call with state What is langchain?
================================ Human Message =================================

What is langchain?
================================== Ai Message ==================================

LangChain is a framework designed to facilitate the development of applications that utilize large language models (LLMs). It provides a set of tools and components that help developers build applications that can interact with LLMs in a more structured and efficient way. LangChain supports various functionalities, including prompt management, memory management, and integration with external data sources, enabling developers to create complex workflows and applications that leverage the capabilities of language models. The framework is particularly useful for task